# Integrate Data Validation Into Your Pipeline



In [ ]:
# Prep environment and logging

import json
import os
import logging
import great_expectations as ge
import great_expectations.jupyter_ux
import pandas as pd
import uuid # used to generate run_id
from datetime import datetime

import tzlocal

great_expectations.jupyter_ux.setup_notebook_logging()



## Integrate data validation into your pipeline

[**Watch a 5 minute tutorial video**](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#video?utm_source=notebook&utm_medium=integrate_validation)


[**Read more in the tutorial**](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html?utm_source=notebook&utm_medium=integrate_validation)

**Reach out for help on** [**Great Expectations Slack**](https://tinyurl.com/great-expectations-slack)




### Get a DataContext object


In [ ]:
context = ge.data_context.DataContext()

### Get a pipeline run id

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#run_id?utm_source=notebook&utm_medium=integrate_validation)


In [ ]:
# Generate a run-id that GE will use to key shared parameters
run_id = datetime.utcnow().isoformat()
run_id

### Choose data asset name and expectation suite name

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#choose_data_asset_and_expectation_suite?utm_source=notebook&utm_medium=integrate_validation)


In [ ]:
context.list_expectation_suites()

In [ ]:
data_asset_name = "REPLACE ME!" # TODO: replace with your value!
expectation_suite_name = "my_suite" # TODO: replace with your value!

### Obtain the batch to validate

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#obtain_a_batch_to_validate?utm_source=notebook&utm_medium=integrate_validation)



##### If your pipeline processes Pandas Dataframes:

```
df = pd.read_csv(file_path_to_validate)
df.head()
batch = context.get_batch(data_asset_name, expectation_suite_name, df)
```

##### If your pipeline processes Spark Dataframes:
```
from pyspark.sql import SparkSession
from great_expectations.dataset import PandasDataset, SqlAlchemyDataset, SparkDFDataset
spark = SparkSession.builder.getOrCreate()
df = SparkDFDataset(spark.read.csv(file_path_to_validate))
df.spark_df.show()
batch = context.get_batch(data_asset_name, expectation_suite_name, df)
```

##### If your pipeline processes SQL querues:
```
batch = context.get_batch(data_asset_name, expectation_suite_name, query="SELECT * from ....") # the query whose result set you want to validate
```


### Validate the batch

This is the "workhorse" method of Great Expectations. Call it in your pipeline code after loading the file and just before passing it to your computation.

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#validate?utm_source=notebook&utm_medium=integrate_validation)



In [ ]:
validation_result = batch.validate(run_id=run_id)

if validation_result["success"]:
    print("This file meets all expectations from a valid batch of {0:s}".format(data_asset_name))
else:
    print("This file is not a valid batch of {0:s}".format(data_asset_name))


### Review the validation results

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#review_validation_results?utm_source=notebook&utm_medium=integrate_validation)


In [ ]:
print(json.dumps(validation_result, indent=4))

### Finishing touches - notifications and saving validation results and validated batches

#### Notifications
You want to be notified when the pipeline validated a batch, especially when the validation failed.

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#send_notifications?utm_source=notebook&utm_medium=integrate_validation)

#### Saving validation results

To enable the storing of validation results, uncomment the `result_store` section in your great_expectations.yml file. 

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#save_validation_results?utm_source=notebook&utm_medium=integrate_validation)

#### Saving failed batches

When a batch fails validation (it does not pass all the expectations of the data asset), it is useful to save the batch along with the validation results for future review. You can enable this option in your project's great_expectations.yml file. 

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/tutorials/pipeline_integration.html#save_failed_batches?utm_source=notebook&utm_medium=integrate_validation)

